In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = r"https://fantasyfootballcalculator.com/scenario-calculator"
params = lambda x,teams,scoring: {'format':scoring,
          'num_teams':teams,
          'draft_pick':x}

probabilities = {}
html = {}
for i in range(1,201):
    test = requests.get(url,params(i,10,'standard'))
    if test.status_code == 200:
        soup = BeautifulSoup(test.text, 'html.parser')
        html[i] = test
        probabilities[i] = soup.select("table")
    else:
        print(f"{i} failed")
        print(test.status_code)

df = pd.read_html(str(soup.find_all('table')[0]))

dfs = None
for k, table in probabilities.items():
    df = pd.read_html(str(table[0]))[0]
    df['pick'] = k
    if dfs is None:
        dfs = df
    else:
        dfs = dfs.append(df)

dfs

,Name,Pos,Team,Bye,%,Unnamed: 5,pick
0,Christian McCaffrey,RB,CAR,13,100.0%,.,1
1,Saquon Barkley,RB,NYG,11,100.0%,.,1
2,Ezekiel Elliott,RB,DAL,10,100.0%,.,1
3,Derrick Henry,RB,TEN,7,100.0%,.,1
4,Alvin Kamara,RB,NO,6,100.0%,.,1
...,...,...,...,...,...,...,...
200,Mason Crosby,PK,GB,5,0.0%,.,200
201,Jake Elliott,PK,PHI,9,0.0%,.,200
202,Dan Bailey,PK,MIN,7,0.0%,.,200
203,Jason Myers,PK,SEA,6,0.0%,.,200


In [7]:
mapping = pd.read_csv('probmap.csv')

In [8]:
mapping

,probname,espnid
0,Christian McCaffrey,3117251
1,Saquon Barkley,3929630
2,Ezekiel Elliott,3051392
3,Derrick Henry,3043078
4,Alvin Kamara,3054850
...,...,...
196,Mason Crosby,10636
197,Jake Elliott,3050478
198,Dan Bailey,14322
199,Jason Myers,2473037


In [10]:
dfs = dfs.merge(mapping, how='left', left_on='Name', right_on='probname')

In [12]:
dfs[['Name','Pos','Team','Bye','%','pick','espnid']].to_pickle('pick_prob_826.pkl')